
### 1. use the Yelp API to search your favorite city for a cuisine type of your choice.

### 2. Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

In [1]:
# Imports
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Functions
def create_json_file(JSON_FILE,  delete_if_exists=False):

    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    ## If it DOES exist:
    if file_exists == True:

        ## Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            


    ## If it does NOT exist:
    else:

        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



### 1. use the Yelp API to search your favorite city for a cuisine type of your choice.

In [3]:
## Yelp Credentials/Accessing the Yelp API
# API Credentials
with open('/Users/Oliver/.secret/yelp_api.json') as f:   
    login = json.load(f)
    
# Instantiating YelpAPI 
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
# setting API call parameters
LOCATION='Winter Springs, FL'
TERM='Burgers'

In [5]:
## Creating results-in-progress json file
# Specifying filename
JSON_FILE= "Data/results_in_progress_FL_pizza.json"
JSON_FILE

'Data/results_in_progress_FL_pizza.json'

In [6]:
## Check if our JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_FL_pizza.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [8]:
# Determining how many pages of results is needed
results=yelp_api.search_query(location=LOCATION,
                             term=TERM,
                             offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

874

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [ ]:
# Adding progress bar in our for loop
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

In [14]:
## For Loop to call each page
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)

print(f'Total number results: {total_results}')
print(f'Total number of pages: {n_pages}')

[!] Data/results_in_progress_FL_pizza.json already exists. Deleting previous file...
[i] Data/results_in_progress_FL_pizza.json not found. Saving empty list to new file.
- 0 previous results found.
Total number results: 874
Total number of pages: 44


### 2. Extract all of the results from your search and compile them into one dataframe


In [16]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/44 [00:00<?, ?it/s]

In [17]:
# Converting .json to DF
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6vtuq0T_Hlq6qsCWIpOpuQ,red-eyes-git-n-messy-smokehouse-and-tavern-win...,Red-Eye's Git N Messy Smokehouse & Tavern,https://s3-media1.fl.yelpcdn.com/bphoto/Ql2pSk...,False,https://www.yelp.com/biz/red-eyes-git-n-messy-...,158,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.5,"{'latitude': 28.7031155136773, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '855 E State Rd 434', 'address2':...",+14075427728,(407) 542-7728,1101.632324
1,xGeQSE9PpMCi1BRaXEHnsQ,general-public-house-winter-springs-2,General Public House,https://s3-media4.fl.yelpcdn.com/bphoto/6kjoZH...,False,https://www.yelp.com/biz/general-public-house-...,234,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 28.69843, 'longitude': -81.26168}",[delivery],$$,"{'address1': '156 Tuskawilla Rd', 'address2': ...",+14078909696,(407) 890-9696,1911.807684
2,pvwGB6dvjR_kqSbsrVTzhQ,fords-garage-oviedo-oviedo,Ford's Garage Oviedo,https://s3-media3.fl.yelpcdn.com/bphoto/LxbMgj...,False,https://www.yelp.com/biz/fords-garage-oviedo-o...,44,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",3.5,"{'latitude': 28.65611, 'longitude': -81.20066}",[],NaN,"{'address1': '459 E Mitchell Hammock Rd', 'add...",+14072133673,(407) 213-3673,9460.446180
3,5bS_zCXfqzoiiEKL07_fJA,grattin-dog-altamonte-springs,Grattin Dog,https://s3-media2.fl.yelpcdn.com/bphoto/bqjQAy...,False,https://www.yelp.com/biz/grattin-dog-altamonte...,82,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 28.662688, 'longitude': -81.357327}","[pickup, delivery]",$,"{'address1': '1006 E Altamonte Dr', 'address2'...",+14079517267,(407) 951-7267,9401.407144
4,oF344ESz0ZbbAG4Aa7wfXw,build-my-burgers-orlando-2,Build My Burgers,https://s3-media1.fl.yelpcdn.com/bphoto/qcSR4X...,False,https://www.yelp.com/biz/build-my-burgers-orla...,147,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 28.598284, 'longitude': -81.22112}","[pickup, delivery]",NaN,"{'address1': '3402 Technological Ave', 'addres...",+14079855495,(407) 985-5495,13485.449245


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
869,7XtbOun49vxy-BIvIwKtjw,ny-pizza-baby-apopka-2,NY Pizza Baby,https://s3-media1.fl.yelpcdn.com/bphoto/QP190_...,False,https://www.yelp.com/biz/ny-pizza-baby-apopka-...,108,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 28.67363, 'longitude': -81.47968}","[delivery, pickup]",$$,"{'address1': '380 Semoran Commerce Pl', 'addre...",+14074647157,(407) 464-7157,20175.840381
870,MGErJxUfrUyUQri8ntcDkg,red-lobster-orlando-2,Red Lobster,https://s3-media2.fl.yelpcdn.com/bphoto/oiYDmr...,False,https://www.yelp.com/biz/red-lobster-orlando-2...,107,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'latitude': 28.55288067, 'longitude': -81.339...","[delivery, pickup]",$$,"{'address1': '3552 E Colonial Dr', 'address2':...",+14078963657,(407) 896-3657,18443.647605
871,YYVftao6UDHDzqeJn0SAaw,red-lobster-orlando-6,Red Lobster,https://s3-media2.fl.yelpcdn.com/bphoto/odbo6d...,False,https://www.yelp.com/biz/red-lobster-orlando-6...,115,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'latitude': 28.55454509, 'longitude': -81.205...","[delivery, pickup]",$$,"{'address1': '617 N Alafaya Trl', 'address2': ...",+14072494141,(407) 249-4141,18550.793795
872,vXaumClW2dQx6w3QlDHx4w,chipotle-mexican-grill-orlando,Chipotle Mexican Grill,https://s3-media3.fl.yelpcdn.com/bphoto/j007hj...,False,https://www.yelp.com/biz/chipotle-mexican-gril...,248,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.0,"{'latitude': 28.5535264892762, 'longitude': -8...","[delivery, pickup]",$,"{'address1': '464 N Alafaya Trl', 'address2': ...",+14072080778,(407) 208-0778,18621.936119
873,7RbWIxU8u4YjiGWZYP25ZQ,chick-fil-a-orlando-13,Chick-fil-A,https://s3-media3.fl.yelpcdn.com/bphoto/fvqEi8...,False,https://www.yelp.com/biz/chick-fil-a-orlando-1...,109,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,"{'latitude': 28.5557522, 'longitude': -81.2062...",[],$,"{'address1': '633 N Alafaya Trl', 'address2': ...",+14072818897,(407) 281-8897,18404.544581


In [18]:
# checking for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [19]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)